In [245]:
import gadata_funcs,pdb,re
import pandas as pd
import numpy as np
import pylab as pl
from gadata_funcs import date2days,gadata_date2days,salesdata_date2days
from importlib import reload
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sys import stdout

%matplotlib notebook

In [21]:
# Loading in data and adding day info
df=pd.read_csv("gadata.csv",dtype={'date':str})
df2=pd.read_csv("/home/wlwoon/Dropbox/chicken_work/impersuasion_stuff/withSalesData/sales_order_line_timestamped.csv")
df['day']=df.date.apply(gadata_date2days)
df2['day']=df2.transaction_date.apply(salesdata_date2days)

In [209]:
# The ranges for the feature generate and target windows respectively
featLen=30
labelLen=20
featThreshold=150

inFeatRange=set(list(range(featThreshold-featLen,featThreshold)))
labelRange=set(list(range(featThreshold,featThreshold+labelLen)))

In [231]:
# Preparing to extract features
dff=df[df.day.isin(inFeatRange)]
dff2=df2[df2.day.isin(labelRange)]
dff2=dff2[dff2.customer_id.isin(set(dff.user_id))]
dg=dff.groupby('user_id')
dff.replace(np.nan,-10,inplace=True)

/home/wlwoon/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [232]:
# Feature extraction

# x (inputs)
xdf=dg.hits.count()
xdf=pd.concat((xdf,dg.country.first()),axis=1)
xdf['malaysia']=(xdf.country=='Malaysia').astype(int)
xdf['malsing']=(xdf.country.isin(['Malaysia','Singapore'])).astype(int)
xdf['meanDay']=dg.day.mean()
xdf['meanDuration']=dg.sessionDuration.mean()
xdf['maxDuration']=dg.sessionDuration.max()
xdf['meanPurchase']=dg.noPurchases.mean()
xdf['maxPurchase']=dg.noPurchases.max()
xdf['recency']=dg.day.apply(lambda tmp: np.min(featThreshold-tmp))

# y (targets)
ydf=pd.DataFrame(xdf.index.isin(set(dff2.customer_id)).astype(int),index=xdf.index)

In [254]:
# Just the relevant features, and formatting for easier use
x=xdf[['hits','maxPurchase']].values
y=ydf.values.reshape((-1,))

In [255]:
scores=[]
importances=[]
print("Cross validation round: ")
for count in range(10):
    stdout.write("#"+str(count)+",")
    
    #rf=GradientBoostingClassifier(n_estimators=300)
    #rf=XGBClassifier(n_estimators=250,max_depth=6)
    rf=RandomForestClassifier(n_estimators=350,max_depth=6)
    
    xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.1)
    rf.fit(xtrain,ytrain)
    importances.append(rf.feature_importances_)
    scores.append([accuracy_score(ytest,rf.predict(xtest)),precision_score(ytest,rf.predict(xtest)),recall_score(ytest,rf.predict(xtest))])

print()
scores=pd.DataFrame(scores,columns=['Accuracy','Precision','Recall'])
print(np.mean(scores))

Cross validation round: 
#0,#1,#2,#3,#4,#5,#6,#7,#8,#9,
Accuracy     0.750353
Precision    0.687475
Recall       0.471838
dtype: float64
